```
[Title] Multivariate Multi Labels LSTM
[Author] Jason Brownlee
[Etidor] Yibeck.Lee(yibeck.lee@gmail.com)
[reference]
 - https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
```

In [3]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
# split a multivariate sequence into samples
from numpy import array
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)



In [8]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
print(out_seq)
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)


[[ 25]
 [ 45]
 [ 65]
 [ 85]
 [105]
 [125]
 [145]
 [165]
 [185]]
[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [9]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)

# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [10]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')


In [11]:
# fit model
model.fit(X, y, epochs=400, verbose=1)


Epoch 1/400
6/6 [==============================] - 4s 654ms/step - loss: 9717.2656
Epoch 2/400
6/6 [==============================] - 0s 7ms/step - loss: 9420.3916
Epoch 3/400
6/6 [==============================] - 0s 3ms/step - loss: 9125.2676
Epoch 4/400
6/6 [==============================] - 0s 2ms/step - loss: 8867.1260
Epoch 5/400
6/6 [==============================] - 0s 6ms/step - loss: 8572.5156
Epoch 6/400
6/6 [==============================] - 0s 4ms/step - loss: 8265.8604
Epoch 7/400
6/6 [==============================] - 0s 3ms/step - loss: 7943.7026
Epoch 8/400
6/6 [==============================] - 0s 3ms/step - loss: 7581.0488
Epoch 9/400
6/6 [==============================] - 0s 2ms/step - loss: 7199.2261
Epoch 10/400
6/6 [==============================] - 0s 4ms/step - loss: 6777.2739
Epoch 11/400
6/6 [==============================] - 0s 4ms/step - loss: 6329.0659
Epoch 12/400
6/6 [==============================] - 0s 5ms/step - loss: 5861.8579
Epoch 13/400
6/6 [=====

In [12]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=1)


1/1 [==============================] - 1s 675ms/step


In [13]:
yhat

array([[100.49953, 105.58044, 206.51749]], dtype=float32)